## Project : Philosophy Bot

data set pdf links: 
https://www.infobooks.org/pdfview/7529-eastern-philosophy-jsrl-narayana-moorty/
https://www.infobooks.org/authors/classic/plato-books/#Republic
https://www.infobooks.org/book/on-youth-old-age-life-and-death-and-respiration-aristotle/
https://www.infobooks.org/pdfview/7225-the-communist-manifesto-karl-marx/


related sources:
https://www.kaggle.com/code/gpreda/rag-using-llama3-langchain-and-chromadb
https://www.kaggle.com/code/vanvalkenberg/nlp-what-the-philosopher-said/notebook

https://www.youtube.com/watch?v=luFHMtaw9pk&ab_channel=DavidBU
https://www.youtube.com/watch?v=2TJxpyO3ei4&t=2s&ab_channel=pixegami
https://www.youtube.com/watch?v=tcqEUSNCn8I&ab_channel=pixegami
https://www.youtube.com/watch?v=Ylz779Op9Pw&ab_channel=ShawTalebi
https://www.youtube.com/watch?v=au2WVVGUvc8&t=307s&ab_channel=LiamOttley